# Создание одномерной общей таблицы финансирования за 2023 год и отдельных таблиц для отчетов и совещаний

## Предыстория.

В нашем отделе, к сожалению, нет никакой автоматизации, однако данных очень много. Сама система очень запутанна, нет установленных форм документов в текущей работе, все отрабатывается вручную, что влечет ошибки человеческого фактора. И это нормально. К тому же такую рутинную работу человек будет выполнять раз в 20-50 медленнее, чем обычный простенький небольшой код, написанный в основном с помощью бибилиотеки панд (хи-хи, Пандаса).

Система в общих словах:

Каждый год планируются лимиты бюджетных обязательств по заданиям ГОЗ, наименованиям закупки и КБК (последнее указывает на то, для кого ЛБО(лимиты) предназначены. Например, учебные заведения или медики. У них свои шифры). В течении года деньги распределяются письмами-распределениями. Мы должны знать:

1. Сколько ЛБО было доведено в конкретное заведение, по каким заданиям ГОЗ и КБК, когда доведены (№ письма);
2. И немаленький блок с разного вида остатками: ЛБО-все распределения с разбивкой по заданиям ГОЗ, наименованиям закупки, КБК (остаки для начальника только по заданиям и КБК в тысячах рублей, остатки для начальника поменьше с заданиями гоз и КБК в рублях, для меня и основной работы в полном разрезе в рублях).

**Основная загвоздка** состоит в том, что письма-распределения практически страшный сон и антипаттерные двухмерные таблички в нескончаемом варианте. Предложения переделать их, к сожалению, руководством было отклонено (смотрят распечатку на А3), поэтому работаем с чем есть и превращаем обработку всего этого массива в 2х минутное радостное занятие с Юпитером.

### Обычный импорт нужных библиотек и создание 2х функций на будущее

In [2]:
import pandas as pd
from numpy import nan
import numpy as np
from os import listdir
from os.path import isfile, join
from itertools import chain
def cleaner(tabl):
    tabl['Задание ГОЗ'] = tabl['Задание ГОЗ'].fillna(method='ffill')
    tabl['Наименование закупки'] = tabl['Наименование закупки'].fillna(method='ffill')
    tabl['КБК'] = tabl['КБК'].fillna(method='ffill')
    tabl['КБК'] = tabl['КБК'].astype('object')
    tabl = tabl.drop(index=[0,1])
    return tabl
def get_tovd_stats(tovds):
    def wrapper(row):
        tovd_stats = []
        for tovd in tovds:
            if row[tovd] != nan:
                tovd_stats.append([tovd, row["КБК"], row["Задание ГОЗ"], row["Наименование закупки"], row[tovd], row["Номер письма"]])
        return tovd_stats
    return wrapper

### Открываем все нужные нам файлы и сохраняем и списком в переменной all_files

Откроем все файлы и почистим их созданной функцией cleaner()

In [3]:
#Переменная для обновления списка (чтобы не задвоилось/троилось)
all_files = []
path = 'Original'
all_name_files = [f for f in listdir(path) if isfile(join(path, f)) and (f.endswith(".xls") or f.endswith(".xlsx"))]

for name in all_name_files:
    source = pd.read_excel(path+"/"+name, header=1)
    del source[source.columns[-1]]
    source = cleaner(source)
    source ['Номер письма'] = name
    all_files.append(source)

### Создаем одномерные таблицы из каждой в списке new_all_files

Просто отработаем каждую двухмерную и страшную табличку написанным циклом с использованием ранее введенной функции get_tovd_stats(). Можно было бы сначала соединить через concat(), но я переживала и решила наглядно проверить, что ничего не напутанно и посчиталось правильно в парочке отдельно взятых таблиц.

In [3]:
new_all_files=[]

for file in all_files:
    t=file
    tovd=t.columns[3:-2]
    t=t.apply(get_tovd_stats(tovd), axis=1)
    t = list(chain.from_iterable(t))
    new_all_files.append(pd.DataFrame(np.array(t),
                                      columns=['Бюджетополучатель', 'КБК', 
                                               'Задание ГОЗ', 'Наименование закупки', 
                                               'Сумма (руб)', '№ письма']))


### Проверка суммы первой таблицы

Та самая первая проверка, которая прошла успешно. Ранее эти 6 млрд рублей сотрудники бы разбивали в удобоваримую таблицу ручками.

In [4]:
new_all_files[0]['Сумма (руб)'] = pd.to_numeric(new_all_files[0]['Сумма (руб)'], errors='coerce')
print(new_all_files[0]['Сумма (руб)'].sum())

6045883611.219999


### Объеденяем таблицы и обрабатываем их (освобождаем от нулей)

Теперь сделаем из всех табличек одну общую "базу", на основе которой проведен все остальные манипуляции.

P.S. Там есть nan объект, надо от него избавиться

In [5]:
cat = pd.concat(new_all_files)
cat['Сумма (руб)'] = pd.to_numeric(cat['Сумма (руб)'], errors = 'coerce')
cat = cat.dropna(subset = 'Сумма (руб)')
cat = cat.reset_index(drop=True)
cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1164 entries, 0 to 1163
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Бюджетополучатель     1164 non-null   object 
 1   КБК                   1164 non-null   object 
 2   Задание ГОЗ           1164 non-null   object 
 3   Наименование закупки  1164 non-null   object 
 4   Сумма (руб)           1164 non-null   float64
 5   № письма              1164 non-null   object 
dtypes: float64(1), object(5)
memory usage: 54.7+ KB


###  Выгружаем базу

Здесь можно посмотреть, в какой территориальный орган сколько лимитов бюджетных обязательств было доведено, на что конкретно, по какому заданию ГОЗ и какому КБК. Сохраняем в специальную папочку рядом с кодом.

In [18]:
with pd.ExcelWriter('.Result\\Финансирование ТОВД, ОУМТС, ФКУ, ДТ 2023.xlsx') as writer:
    cat.to_excel(writer)

# ВТОРАЯ ЧАСТЬ. ОСТАТКИ

Просто так деньги ниоткуда не берутся, кроме того они не бесконечны. Значит, есть их лимиты и больше этих лимитов мы дать не можем. Наши деньги разложены по "кармашкам", заданиям ГОЗ, наименованиям закупки и КБК, которые мы запланировали давно-давно, при формировании. Около года назад. И как приличные планировщики должны в рамках этого и работать. Прочитаем табличку с лимитами.

In [7]:
lbo = pd.read_excel('ЛБО 23.xlsx')
lbo['Задание ГОЗ'] = lbo['Задание ГОЗ'].fillna(method='ffill')
lbo['Наименование закупки'] = lbo['Наименование закупки'].fillna(method='ffill')
lbo['КБК'] = lbo['КБК'].fillna(method='ffill')
lbo['КБК'] = lbo['КБК'].astype('object')
lbo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315 entries, 0 to 314
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Задание ГОЗ           315 non-null    object 
 1   Наименование закупки  315 non-null    object 
 2   КБК                   315 non-null    object 
 3   ЛБО                   315 non-null    float64
dtypes: float64(1), object(3)
memory usage: 10.0+ KB


Поработаем с нашими базовыми данными-распределениями.

In [8]:
for_tail = cat.copy()
del for_tail[for_tail.columns[0]]
for_tail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1164 entries, 0 to 1163
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   КБК                   1164 non-null   object 
 1   Задание ГОЗ           1164 non-null   object 
 2   Наименование закупки  1164 non-null   object 
 3   Сумма (руб)           1164 non-null   float64
 4   № письма              1164 non-null   object 
dtypes: float64(1), object(4)
memory usage: 45.6+ KB


Во-первых, соединим лимиты и уже распределенные лбо (лимиты бюджетных обязательств) в одну таблицу. Во-вторых, для упрощения математики и понимания, все распределения превратим в отрицательные числа. Логика проста: есть лимиты, а есть то, что их уменьшает-наши распределения.

In [9]:
#tail = pd.concat(for_tail, keys = ['Задание ГОЗ', 'Наименование закупки', 'КБК'])[lbo] ignore_index
tail = pd.concat([lbo, for_tail])
tail['Сумма (руб)'] = tail['Сумма (руб)']*-1
tail.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1479 entries, 0 to 1163
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Задание ГОЗ           1479 non-null   object 
 1   Наименование закупки  1479 non-null   object 
 2   КБК                   1479 non-null   object 
 3   ЛБО                   315 non-null    float64
 4   Сумма (руб)           1164 non-null   float64
 5   № письма              1164 non-null   object 
dtypes: float64(2), object(4)
memory usage: 80.9+ KB


А теперь моя самая любимая часть! Воспользуемся pivot_table(). У кого-то есть любимое время года или цвет, а у меня-любимый инструмент обработки данных. Его придумал гений.

In [10]:
tail = tail.pivot_table(index=['Задание ГОЗ', 'Наименование закупки', 'КБК'], values=['ЛБО', 'Сумма (руб)'], aggfunc='sum')

In [11]:
tail

ЛБО  \
Задание ГОЗ                                        Наименование закупки                               КБК                                
Бумагоуничтожающая машина                          Бумагоуничтожающая машина (Тип 1)                  0302 0830292019 211    6593000.0   
                                                   Бумагоуничтожающая машина (Тип 2)                  0302 0830292019 211    6084000.0   
                                                   Бумагоуничтожающая машина (Тип 3)                  0302 0830292019 211    3018400.0   
                                                   Бумагоуничтожающая машина (Тип 4)                  0302 0830292019 211    5106000.0   
Источник бесперебойного питания                    ИБП для ПЭВМ                                       0302 0830292019 211   67837100.0   
...                                                                                                                                ...   
Шифровальные (криптографические) средства согла... 10% на ХВ1000                                      0302 0830292019 211   22750000.0   
                                                   Континенты для новых территорий ЗМЗ                0302 0830292019 211   27620300.0   
                                                   Поставка СКЗИ М-479Р3К                             0302 0830292019 211    5850000.0   
                                                   Поставка программно-аппаратных комплексов крипт... 0302 0830292019 211  227500000.0   
                                                   Средства криптографической защиты информации М-... 0302 0830292019 211          0.0   

                                                                                                                            Сумма (руб)  
Задание ГОЗ                                        Наименование закупки                               КБК                                
Бумагоуничтожающая машина                          Бумагоуничтожающая машина (Тип 1)                  0302 0830292019 211 -6.641423e+06  
                                                   Бумагоуничтожающая машина (Тип 2)                  0302 0830292019 211 -6.042110e+06  
                                                   Бумагоуничтожающая машина (Тип 3)                  0302 0830292019 211 -3.019072e+06  
                                                   Бумагоуничтожающая машина (Тип 4)                  0302 0830292019 211 -5.098747e+06  
Источник бесперебойного питания                    ИБП для ПЭВМ                                       0302 0830292019 211 -7.159256e+07  
...                                                                                                                                 ...  
Шифровальные (криптографические) средства согла... 10% на ХВ1000                                      0302 0830292019 211  0.000000e+00  
                                                   Континенты для новых территорий ЗМЗ                0302 0830292019 211  0.000000e+00  
                                                   Поставка СКЗИ М-479Р3К                             0302 0830292019 211 -1.170000e+07  
                                                   Поставка программно-аппаратных комплексов крипт... 0302 0830292019 211 -2.275000e+08  
                                                   Средства криптографической защиты информации М-... 0302 0830292019 211  5.850000e+06  

[380 rows x 2 columns]

Это-моя основная таблица для работы. Мне нужно все, даже копейки. Выгружаю ее для себя.

In [12]:
with pd.ExcelWriter('.Result\\Остатки с наименованием закупки 2023.xlsx') as writer:
    tail.to_excel(writer)

Теперь сделаем такую же, но исключительно по заданиям ГОЗ (это для начальника поменьше, который хочет знать копейки, но не хочет копаться в наименованиях закупки).

In [13]:
tail1 = tail.copy()
tail1 = tail1.pivot_table(index=['Задание ГОЗ', 'КБК'], values=['ЛБО', 'Сумма (руб)'], aggfunc='sum')
tail1['Остатки']= tail1['ЛБО']+tail1['Сумма (руб)']

In [20]:
tail1

,,ЛБО,Сумма (руб),Остатки
Задание ГОЗ,КБК,,,
Бумагоуничтожающая машина,0302 0830292019 211,2.080140e+07,-2.080135e+07,4.753000e+01
Источник бесперебойного питания,0302 0830292019 211,1.350468e+08,-1.350468e+08,-3.851000e+01
"Комплект средств глобальной навигационной спутниковой системы ""ГЛОНАСС"" согласно перечню, утвержденному МВД России",0302 0830292019 211,2.757276e+08,-1.976129e+08,7.811466e+07
"Комплекты средств для проведения контроля и оценки защищенности согласно перечню, утвержденному МВД России",0302 0830292019 211,1.153143e+08,-1.153142e+08,5.000000e+01
Копировально-множительный аппарат,0302 0830292019 211,9.059480e+07,-9.059484e+07,-4.300000e+01
Ламинатор,0302 0830292019 211,4.367900e+06,-4.367921e+06,-2.058000e+01
Многофункциональное печатающее устройство,0302 0830292019 211,2.404327e+08,-2.404327e+08,-3.200000e+01
"Оборудование для предоставления государственных услуг согласно перечню, утвержденному МВД России",0302 0830292019 211,6.366490e+07,-6.366492e+07,-1.998000e+01
"Оборудование телекоммуникационных инфраструктур согласно перечню, утвержденного МВД России",0302 0830292019 211,1.089610e+07,-1.089613e+07,-3.000000e+01


In [15]:
with pd.ExcelWriter('.Result\\Остатки по заданиям ГОЗ 2023.xlsx') as writer:
    tail1.to_excel(writer)

А теперь сделаем табличку для самого главного начальника, который охватывает все сферы деятельности Департамента, а потому и таличка ему нужна максимально небольшая и информативная (в тысячах рублей).

In [22]:
tail2=tail1.copy()
tail2=tail2/1000
tail2 = tail2.drop('ЛБО', axis=1)
tail2 = tail2.drop('Сумма (руб)', axis=1)
tail2

,,Остатки
Задание ГОЗ,КБК,
Бумагоуничтожающая машина,0302 0830292019 211,0.04753
Источник бесперебойного питания,0302 0830292019 211,-0.03851
"Комплект средств глобальной навигационной спутниковой системы ""ГЛОНАСС"" согласно перечню, утвержденному МВД России",0302 0830292019 211,78114.65700
"Комплекты средств для проведения контроля и оценки защищенности согласно перечню, утвержденному МВД России",0302 0830292019 211,0.05000
Копировально-множительный аппарат,0302 0830292019 211,-0.04300
Ламинатор,0302 0830292019 211,-0.02058
Многофункциональное печатающее устройство,0302 0830292019 211,-0.03200
"Оборудование для предоставления государственных услуг согласно перечню, утвержденному МВД России",0302 0830292019 211,-0.01998
"Оборудование телекоммуникационных инфраструктур согласно перечню, утвержденного МВД России",0302 0830292019 211,-0.03000


In [17]:
with pd.ExcelWriter('.Result\\Остатки по заданиям ГОЗ в тыс руб 2023.xlsx') as writer:
    tail2.to_excel(writer)